# Crypto Protocol Research Dashboard

This notebook fetches real-time data for Syrup (Syrup.fi), MetDAO (Metronome), and MYX (MYX.finance) to build a research summary.

In [1]:
%pip install requests pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.9 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 38.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 47.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.7 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.7 MB/s  0:00:00
   ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/10 [numpy]  WARNING: The scripts f2py and numpy-config are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━  5/10 [fonttools]  WARNING: The scripts fonttools, pyftmerge, pyftsubset and ttx are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PAT

In [2]:
import requests
import pandas as pd
import json

def get_defillama_protocol(slug):
    try:
        url = f"https://api.llama.fi/protocol/{slug}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching {slug}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception fetching {slug}: {e}")
        return None

def search_protocols(query):
    try:
        url = "https://api.llama.fi/protocols"
        response = requests.get(url)
        protocols = response.json()
        return [p for p in protocols if query.lower() in p['name'].lower() or query.lower() in p.get('slug', '').lower()]
    except Exception as e:
        print(f"Error searching: {e}")
        return []

print("Setup complete.")

Setup complete.


In [5]:
# Search for protocols
syrup_candidates = search_protocols("syrup")
myx_candidates = search_protocols("myx")
met_candidates = search_protocols("metronome")

print("Syrup Candidates:", [(p['name'], p['slug']) for p in syrup_candidates])
print("MYX Candidates:", [(p['name'], p['slug']) for p in myx_candidates])
print("Metronome Candidates:", [(p['name'], p['slug']) for p in met_candidates])

Syrup Candidates: [('Syrup Finance', 'syrup-finance')]
MYX Candidates: [('MYX Finance', 'myx-finance')]
Metronome Candidates: [('Metronome Synth', 'metronome-synth'), ('Metronome V1', 'metronome-v1')]


In [9]:
# Fetch details
syrup_data = get_defillama_protocol("syrup-finance")
myx_data = get_defillama_protocol("myx-finance")
met_data = get_defillama_protocol("metronome-synth")

def print_summary(data):
    if not data:
        print("No data")
        return
    print(f"Name: {data.get('name')}")
    desc = data.get('description', '')
    if desc:
        print(f"Description: {desc[:200]}...")
    print(f"URL: {data.get('url')}")
    
    tvl_info = data.get('currentChainTvls', {})
    total_tvl = sum(tvl_info.values()) if tvl_info else 0
    print(f"Current TVL: ${total_tvl:,.2f}")

print("--- Syrup ---")
print_summary(syrup_data)
print("\n--- MYX ---")
print_summary(myx_data)
print("\n--- Metronome ---")
print_summary(met_data)

--- Syrup ---
Name: Syrup Finance
Description: Fully Decentralized Spot & Perpetual Exchange. Go long, go short or swap crypto in self-custody, with up to 100x more buying power...
URL: https://syrup.finance/
Current TVL: $2,456.92

--- MYX ---
Name: MYX Finance
Description: A decentralized derivatives protocol with a Matching Pool Mechanism...
URL: https://app.myx.finance/referrals?invitationCode=H43P6XB
Current TVL: $33,831,518.34

--- Metronome ---
Name: Metronome Synth
Description: Metronome Synth is a leading synthetic protocol that enables users to post multi-collateral assets from their existing crypto holdings and create a multitude of synthetic assets in one complete platfo...
URL: https://www.metronome.io/synth
Current TVL: $22,653,813.14


In [11]:
# Deep dive into Syrup and Metronome
import requests
from bs4 import BeautifulSoup # If available, otherwise regex

def get_html_title(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            # Simple string search for title
            content = response.text
            start = content.find('<title>')
            end = content.find('</title>')
            if start != -1 and end != -1:
                return content[start+7:end]
            return "No title found"
        return f"Error {response.status_code}"
    except Exception as e:
        return f"Exception: {e}"

print("Syrup.fi Title:", get_html_title("https://syrup.fi"))
print("Metronome.io Title:", get_html_title("https://metronome.io"))

# Search for Maple
maple_candidates = search_protocols("maple")
print("Maple Candidates:", [(p['name'], p['slug']) for p in maple_candidates])

# Check for MIP-12 in Metronome governance (if we can guess URL)
# Try to fetch snapshot space?
# Or just search for "MIP-12" in metronome.io content?
met_content = requests.get("https://metronome.io", headers={'User-Agent': 'Mozilla/5.0'}).text
if "MIP-12" in met_content:
    print("Found MIP-12 in Metronome homepage")
else:
    print("MIP-12 not found in Metronome homepage")

Syrup.fi Title: Maple Finance
Metronome.io Title: DeFi’s Leading Synthetic Protocol | Metronome
Maple Candidates: [('Maple', 'maple'), ('MapleDeFi', 'mapledefi'), ('Maple RWA', 'maple-rwa')]
MIP-12 not found in Metronome homepage


In [13]:
# Fetch Maple and MYX Volume
maple_data = get_defillama_protocol("maple")
maple_rwa_data = get_defillama_protocol("maple-rwa")

print("--- Maple ---")
print_summary(maple_data)
print("--- Maple RWA ---")
print_summary(maple_rwa_data)

# Fetch MYX Volume
try:
    # This endpoint returns all DEXs, we filter for MYX
    dex_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
    dex_resp = requests.get(dex_url).json()
    myx_vol_data = next((d for d in dex_resp.get('protocols', []) if d.get('slug') == 'myx-finance'), None)
    
    if myx_vol_data:
        print("\n--- MYX Volume ---")
        print(f"Daily Volume: ${myx_vol_data.get('dailyVolume', 0):,.2f}")
        print(f"Total Volume: ${myx_vol_data.get('totalVolume24h', 0):,.2f}")
    else:
        print("\nMYX Volume not found in DEX overview")
except Exception as e:
    print(f"Error fetching DEX volume: {e}")

# Check Metronome Governance URLs
gov_urls = ["https://forum.metronome.io", "https://vote.metronome.io", "https://governance.metronome.io"]
print("\n--- Metronome Governance Check ---")
for url in gov_urls:
    try:
        r = requests.head(url, timeout=3)
        print(f"{url}: {r.status_code}")
    except:
        print(f"{url}: Failed")

--- Maple ---
Name: Maple
Description: Maple Finance is an institutional capital marketplace powered by blockchain technology....
URL: https://www.maple.finance
Current TVL: $6,256,671,207.86
--- Maple RWA ---
Name: Maple RWA
Description: Maple Finance is an institutional capital marketplace powered by blockchain technology....
URL: https://www.maple.finance
Current TVL: $35,629,890.26

MYX Volume not found in DEX overview

--- Metronome Governance Check ---
https://forum.metronome.io: Failed
https://vote.metronome.io: Failed
https://governance.metronome.io: Failed


In [ ]:
# Fetch Syrup Meta and MYX Volume Search
def get_meta_description(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            meta = soup.find('meta', attrs={'name': 'description'})
            if meta:
                return meta.get('content')
            # Try og:description
            meta = soup.find('meta', attrs={'property': 'og:description'})
            if meta:
                return meta.get('content')
            return "No description found"
        return f"Error {response.status_code}"
    except Exception as e:
        return f"Exception: {e}"

# Need bs4
try:
    from bs4 import BeautifulSoup
except ImportError:
    print("bs4 not installed, installing...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "beautifulsoup4"])
    from bs4 import BeautifulSoup

print("Syrup.fi Description:", get_meta_description("https://syrup.fi"))

# Search DEXs for MYX
try:
    dex_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
    dex_resp = requests.get(dex_url).json()
    all_dexs = dex_resp.get('protocols', [])
    myx_found = [d for d in all_dexs if 'myx' in d.get('name', '').lower() or 'myx' in d.get('slug', '').lower()]
    
    print("\n--- MYX DEX Search ---")
    for d in myx_found:
        print(f"Name: {d.get('name')}, Slug: {d.get('slug')}, Daily Vol: {d.get('dailyVolume')}")
except Exception as e:
    print(f"Error searching DEXs: {e}")